In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

project_name = 'hp_search_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=hp_search_para_wordembed


### HP search

In [4]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/bart/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/hyperparameters_test"'

log_dir = output_dir + '/logs'

In [6]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path, use_cache=False)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

In [7]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_name_or_path \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--hp_search \
--fp16 \
--freeze_embeds --freeze_encoder \
--lr_scheduler constant \
--sortish_sampler \
--task summarization \
--max_source_length 512 \
--max_target_length 60 \
--val_max_target_length 60 \
--num_train_epochs 1 \
--n_train 50 \
--n_val 10 \
--logging_steps 1 --logging_first_step \
--per_device_train_batch_size 1 --per_device_eval_batch_size 1 \
--evaluation_strategy steps --eval_steps 3 \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/03/2021 15:03:21 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/03/2021 15:03:21 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/hyperparameters_test', overwrite_output_dir=True, do_train=False, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb03_15-03-21_ip-172-31-39-35', logging_first_step=True, logging_steps=1, save_steps=500, save_total_limit=None, no_cuda

[INFO|modeling_utils.py:1143] 2021-02-03 15:03:51,206 >> All model checkpoint weights were used when initializing BartForConditionalGeneration.

[INFO|modeling_utils.py:1152] 2021-02-03 15:03:51,206 >> All the weights of BartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-cnn-12-6.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.
02/03/2021 15:03:51 - INFO - utils -   setting model.config to task specific params for summarization:
 {'early_stopping': True, 'length_penalty': 2.0, 'max_length': 142, 'min_length': 56, 'no_repeat_ngram_size': 3, 'num_beams': 4}
02/03/2021 15:03:51 - INFO - utils -   note: command line args may override some of these
[INFO|modeling_utils.py:1027] 2021-02-03 15:03:51,761 >> loading weights file https://huggingface.co/sshleifer/distilbart-cnn-12-6/resolve/main/pytorch_model.bin from cache at /hom

 33%|███▎      | 4/12 [00:09<00:27,  3.41s/it]
(pid=3369) {'loss': 2.9396, 'learning_rate': 1e-05, 'epoch': 0.32}
 42%|████▏     | 5/12 [00:10<00:15,  2.28s/it]
(pid=3369) {'loss': 4.3526, 'learning_rate': 1e-05, 'epoch': 0.4}
 50%|█████     | 6/12 [00:10<00:09,  1.60s/it]
(pid=3369) {'loss': 3.207, 'learning_rate': 1e-05, 'epoch': 0.48}
(pid=3369) 
  0%|          | 0/10 [00:00<?, ?it/s]
(pid=3369) 
 20%|██        | 2/10 [00:00<00:02,  2.95it/s]
(pid=3369) 
 30%|███       | 3/10 [00:01<00:03,  2.05it/s]
(pid=3369) 
 40%|████      | 4/10 [00:02<00:03,  1.72it/s]
(pid=3369) 
 50%|█████     | 5/10 [00:02<00:03,  1.62it/s]
(pid=3369) 
 60%|██████    | 6/10 [00:03<00:02,  1.57it/s]
(pid=3369) 
 70%|███████   | 7/10 [00:04<00:01,  1.54it/s]
(pid=3369) 
 80%|████████  | 8/10 [00:04<00:01,  1.52it/s]
(pid=3369) 
 90%|█████████ | 9/10 [00:05<00:00,  1.50it/s]
(pid=3369) 
100%|██████████| 10/10 [00:06<00:00,  1.49it/s]
Result for _inner_1aeff_00000:
  date: 2021-02-03_15-05-12
  done: false
  ep

 92%|█████████▏| 11/12 [00:26<00:02,  2.55s/it]
(pid=3369) {'loss': 3.0125, 'learning_rate': 1e-05, 'epoch': 0.88}
100%|██████████| 12/12 [00:26<00:00,  1.86s/it]
(pid=3369) {'loss': 3.2749, 'learning_rate': 1e-05, 'epoch': 0.96}
(pid=3369) 
(pid=3369) 
  0%|          | 0/10 [00:00<?, ?it/s]
(pid=3369) 
 20%|██        | 2/10 [00:00<00:02,  2.95it/s]
(pid=3369) 
 30%|███       | 3/10 [00:01<00:03,  2.07it/s]
(pid=3369) 
 40%|████      | 4/10 [00:02<00:03,  1.73it/s]
(pid=3369) 
 50%|█████     | 5/10 [00:02<00:03,  1.63it/s]
(pid=3369) 
 60%|██████    | 6/10 [00:03<00:02,  1.57it/s]
(pid=3369) 
 70%|███████   | 7/10 [00:04<00:01,  1.54it/s]
(pid=3369) 
 80%|████████  | 8/10 [00:04<00:01,  1.52it/s]
(pid=3369) 
 90%|█████████ | 9/10 [00:05<00:00,  1.51it/s]
(pid=3369) 
100%|██████████| 10/10 [00:06<00:00,  1.50it/s]
Result for _inner_1aeff_00000:
  date: 2021-02-03_15-05-28
  done: false
  epoch: 0.96
  eval_gen_len: 60.0
  eval_loss: 2.467834949493408
  eval_rouge1_fmeasure: 0.3174
  eva

(pid=3368) wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)
(pid=3368) wandb: wandb version 0.10.17 is available!  To upgrade, please run:
(pid=3368) wandb:  $ pip install wandb --upgrade
(pid=3368) wandb: Tracking run with wandb version 0.10.14
(pid=3368) wandb: Syncing run /home/ubuntu/s3/fine-tuning/hyperparameters_test
(pid=3368) wandb: ⭐️ View project at https://wandb.ai/marcoabrate/hp_search_para_wordembed
(pid=3368) wandb: 🚀 View run at https://wandb.ai/marcoabrate/hp_search_para_wordembed/runs/ja04x0p3
(pid=3368) wandb: Run data is saved locally in /home/ubuntu/ray_results/_inner_2021-02-03_15-04-25/_inner_1aeff_00001_1_gradient_accumulation_steps=8,learning_rate=0.0001_2021-02-03_15-05-28/wandb/run-20210203_150558-ja04x0p3
(pid=3368) wandb: Run `wandb offline` to turn off syncing.
  0%|          | 0/6 [00:00<?, ?it/s]
(pid=3368) 
(pid=3368) /home/ubuntu/miniconda3/envs/magma/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:136: U

...

(pid=3370) 
 40%|████      | 4/10 [00:02<00:03,  1.67it/s]
(pid=3370) 
 50%|█████     | 5/10 [00:02<00:03,  1.57it/s]
(pid=3370) 
 60%|██████    | 6/10 [00:03<00:02,  1.52it/s]
(pid=3370) 
 70%|███████   | 7/10 [00:04<00:02,  1.50it/s]
(pid=3370) 
 80%|████████  | 8/10 [00:04<00:01,  1.48it/s]
(pid=3370) 
 90%|█████████ | 9/10 [00:05<00:00,  1.46it/s]
(pid=3370) 
100%|██████████| 10/10 [00:06<00:00,  1.45it/s]
Result for _inner_1aeff_00002:
  date: 2021-02-03_15-06-57
  done: false
  epoch: 0.24
  eval_gen_len: 59.8
  eval_loss: 2.5498046875
  eval_rouge1_fmeasure: 0.3174
  eval_rouge1_precision: 0.2654
  eval_rouge1_recall: 0.4285
  eval_rouge2_fmeasure: 0.1462
  eval_rouge2_precision: 0.1205
  eval_rouge2_recall: 0.2037
  eval_rougeL_fmeasure: 0.2561
  eval_rougeL_precision: 0.2123
  eval_rougeL_recall: 0.3535
  eval_rougeLsum_fmeasure: 0.2504
  eval_rougeLsum_precision: 0.2059
  eval_rougeLsum_recall: 0.343
  eval_runtime: 7.5998
  eval_samples_per_second: 1.316
  eval_sentence_dist

(pid=3370) 
  0%|          | 0/10 [00:00<?, ?it/s]
(pid=3370) 
 20%|██        | 2/10 [00:00<00:02,  2.85it/s]
(pid=3370) 
 30%|███       | 3/10 [00:01<00:03,  1.99it/s]
(pid=3370) 
 40%|████      | 4/10 [00:02<00:03,  1.67it/s]
(pid=3370) 
 50%|█████     | 5/10 [00:02<00:03,  1.58it/s]
(pid=3370) 
 60%|██████    | 6/10 [00:03<00:02,  1.54it/s]
(pid=3370) 
 70%|███████   | 7/10 [00:04<00:01,  1.51it/s]
(pid=3370) 
 80%|████████  | 8/10 [00:04<00:01,  1.48it/s]
(pid=3370) 
 90%|█████████ | 9/10 [00:05<00:00,  1.47it/s]
(pid=3370) 
100%|██████████| 10/10 [00:06<00:00,  1.46it/s]
Result for _inner_1aeff_00002:
  date: 2021-02-03_15-07-14
  done: false
  epoch: 0.72
  eval_gen_len: 59.7
  eval_loss: 2.4825830459594727
  eval_rouge1_fmeasure: 0.3324
  eval_rouge1_precision: 0.2729
  eval_rouge1_recall: 0.4666
  eval_rouge2_fmeasure: 0.1713
  eval_rouge2_precision: 0.1423
  eval_rouge2_recall: 0.2368
  eval_rougeL_fmeasure: 0.2758
  eval_rougeL_precision: 0.2266
  eval_rougeL_recall: 0.3908
 

(pid=3370) wandb: Waiting for W&B process to finish, PID 3962
(pid=3370) wandb: Program ended successfully.
(pid=3370) wandb: - 0.04MB of 0.04MB uploaded (0.00MB deduped)
wandb:                                                                                
(pid=3370) wandb: Find user logs for this run at: /home/ubuntu/ray_results/_inner_2021-02-03_15-04-25/_inner_1aeff_00002_2_gradient_accumulation_steps=4,learning_rate=0.0001_2021-02-03_15-06-18/wandb/run-20210203_150647-2wx99tx3/logs/debug.log
(pid=3370) wandb: Find internal logs for this run at: /home/ubuntu/ray_results/_inner_2021-02-03_15-04-25/_inner_1aeff_00002_2_gradient_accumulation_steps=4,learning_rate=0.0001_2021-02-03_15-06-18/wandb/run-20210203_150647-2wx99tx3/logs/debug-internal.log
(pid=3370) wandb: Run summary:
(pid=3370) wandb:                           train/loss 3.0884
(pid=3370) wandb:                  train/learning_rate 0.0001
(pid=3370) wandb:                          train/epoch 0.96
(pid=3370) wandb:         

 33%|███▎      | 4/12 [00:09<00:26,  3.32s/it]
(pid=3367) {'loss': 2.9396, 'learning_rate': 1e-05, 'epoch': 0.32}
 42%|████▏     | 5/12 [00:09<00:15,  2.23s/it]
(pid=3367) {'loss': 4.3526, 'learning_rate': 1e-05, 'epoch': 0.4}
 50%|█████     | 6/12 [00:09<00:09,  1.56s/it]
(pid=3367) {'loss': 3.207, 'learning_rate': 1e-05, 'epoch': 0.48}
(pid=3367) 
  0%|          | 0/10 [00:00<?, ?it/s]
(pid=3367) 
 20%|██        | 2/10 [00:00<00:02,  2.90it/s]
(pid=3367) 
 30%|███       | 3/10 [00:01<00:03,  2.03it/s]
(pid=3367) 
 40%|████      | 4/10 [00:02<00:03,  1.69it/s]
(pid=3367) 
 50%|█████     | 5/10 [00:02<00:03,  1.60it/s]
(pid=3367) 
 60%|██████    | 6/10 [00:03<00:02,  1.54it/s]
(pid=3367) 
 70%|███████   | 7/10 [00:04<00:01,  1.51it/s]
(pid=3367) 
 80%|████████  | 8/10 [00:04<00:01,  1.49it/s]
(pid=3367) 
 90%|█████████ | 9/10 [00:05<00:00,  1.48it/s]
(pid=3367) 
100%|██████████| 10/10 [00:06<00:00,  1.47it/s]
Result for _inner_1aeff_00003:
  date: 2021-02-03_15-08-08
  done: false
  ep

 83%|████████▎ | 10/12 [00:25<00:07,  3.62s/it]
(pid=3367) {'loss': 3.7499, 'learning_rate': 1e-05, 'epoch': 0.8}
 92%|█████████▏| 11/12 [00:25<00:02,  2.60s/it]
(pid=3367) {'loss': 3.0125, 'learning_rate': 1e-05, 'epoch': 0.88}
100%|██████████| 12/12 [00:26<00:00,  1.89s/it]
(pid=3367) {'loss': 3.2749, 'learning_rate': 1e-05, 'epoch': 0.96}
(pid=3367) 
  0%|          | 0/10 [00:00<?, ?it/s]
(pid=3367) 
 20%|██        | 2/10 [00:00<00:02,  2.91it/s]
(pid=3367) 
 30%|███       | 3/10 [00:01<00:03,  2.02it/s]
(pid=3367) 
 40%|████      | 4/10 [00:02<00:03,  1.68it/s]
(pid=3367) 
 50%|█████     | 5/10 [00:02<00:03,  1.59it/s]
(pid=3367) 
 60%|██████    | 6/10 [00:03<00:02,  1.53it/s]
^C
(pid=3367) 
 70%|███████   | 7/10 [00:04<00:01,  1.51it/s]
